In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

from sklearn.linear_model import LinearRegression

C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv('Big Mart sale Prediction\\train_XnW6LSF.csv')

In [68]:
df_test = pd.read_csv('Big Mart sale Prediction\\test_FewQE9B.csv')

In [4]:
df_sub = pd.read_csv('Big Mart sale Prediction\\sample_submission_hP4II7x.csv')

In [ ]:
df_sub.head()

In [ ]:
df_sub=df_sub.drop(['Item_Outlet_Sales'], axis = 1)

In [ ]:
df_sub.head()

In [5]:
df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [69]:
df_test.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [70]:
df_test.isnull().sum()

Item_Identifier                 0
Item_Weight                   976
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  1606
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [71]:
df_test['Item_Weight'].fillna(df_test['Item_Weight'].median(), inplace = True)

In [72]:
df_test['Outlet_Size'].fillna(df_test['Outlet_Size'].mode()[0], inplace = True)

In [73]:
df_test['Item_Type_Combined'] = df_test['Item_Identifier'].apply(lambda df: df[0:2])
df_test['Item_Type_Combined'] = df_test['Item_Type_Combined'].map({'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'})

df_test['Item_Type_Combined'].value_counts()

Food              4076
Non-Consumable    1087
Drinks             518
Name: Item_Type_Combined, dtype: int64

In [74]:
df_test['Outlet_Years'] = 2013 - df_test['Outlet_Establishment_Year']

In [75]:
df_test['Item_Fat_Content'] = df_test['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
df_test['Item_Fat_Content'].value_counts()

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64

In [76]:
le_1 = LabelEncoder()
df_test['Outlet'] = le.fit_transform(df_test['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le_1 = LabelEncoder()
for i in var_mod:
    df_test[i] = le_1.fit_transform(df_test[i])

In [77]:
df_test = pd.get_dummies(df_test, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [78]:
# dropping the ID variables and variables that have been used to extract new variables
df_test.drop(['Item_Type','Outlet_Establishment_Year', 'Item_Identifier', 'Outlet_Identifier'],axis=1,inplace=True)

In [79]:
df_test.shape

(5681, 29)

In [80]:
Noramlised_df_test = preprocessing.normalize(df_test)

In [81]:
df_test.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,20.750,0.007565,107.8622,14,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,8.300,0.038428,87.3198,6,0,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,14.600,0.099575,241.7538,15,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
3,7.315,0.015388,155.0340,6,1,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,12.500,0.118599,234.2300,28,0,1,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [10]:
df.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

In [7]:
df['Outlet_Type'].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

In [8]:
df['Item_Weight'].fillna(df['Item_Weight'].median(), inplace = True)

In [9]:
df['Outlet_Size'].fillna(df['Outlet_Size'].mode()[0], inplace = True)

In [ ]:
df['Item_Identifier'].nunique()

In [ ]:
df.head()

In [11]:
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda df: df[0:2])
df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'})

df['Item_Type_Combined'].value_counts()

Food              6125
Non-Consumable    1599
Drinks             799
Name: Item_Type_Combined, dtype: int64

In [12]:
# operating years of the store
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']

In [13]:
# modifying categories of Item_Fat_Content
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
df['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [14]:
# label encoding the ordinal variables
le = LabelEncoder()
df['Outlet'] = le.fit_transform(df['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])

In [15]:
# one hot encoding the remaining categorical variables 
df = pd.get_dummies(df, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [16]:
# dropping the ID variables and variables that have been used to extract new variables
df.drop(['Item_Type','Outlet_Establishment_Year', 'Item_Identifier', 'Outlet_Identifier'],axis=1,inplace=True)

In [34]:
normalized_X = preprocessing.normalize(X)

In [32]:
X = df.drop('Item_Outlet_Sales',1)
y = df['Item_Outlet_Sales']

In [50]:
X.shape,y.shape

((8523, 29), (8523,))

In [51]:
X.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Years,Item_Fat_Content_0,Item_Fat_Content_1,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,14,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,4,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,14,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,15,0,1,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,26,1,0,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0


In [39]:
# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

In [22]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import InputLayer, Dense 
from keras.optimizers import SGD

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\test_env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\test_env

In [40]:
input_neurons = X_train.shape[1]

In [85]:
output_neurons =1

In [86]:
number_of_hidden_layers = 2
neuron_hidden_layer_1 = 10
neuron_hidden_layer_2 = 5
neuron_hidden_layer_3 = 2

In [87]:
model = Sequential()
model.add(InputLayer(input_shape=(input_neurons,)))
model.add(Dense(units=neuron_hidden_layer_1, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_2, activation='relu'))
model.add(Dense(units=neuron_hidden_layer_3, activation='relu'))
model.add(Dense(units=output_neurons, activation='linear'))

In [88]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                300       
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 12        
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 3         
Total params: 370
Trainable params: 370
Non-trainable params: 0
_________________________________________________________________


In [89]:
model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['mse'])

In [90]:
model_history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50)

Train on 6392 samples, validate on 2131 samples
Epoch 1/50
6392/6392 [==============================] - 0s 68us/step - loss: 7480188.9462 - mse: 7480188.5000 - val_loss: 6221674.0798 - val_mse: 6221672.0000
Epoch 2/50
6392/6392 [==============================] - 0s 44us/step - loss: 5063556.7243 - mse: 5063556.5000 - val_loss: 2643638.2642 - val_mse: 2643638.0000
Epoch 3/50
6392/6392 [==============================] - 0s 42us/step - loss: 2196326.2279 - mse: 2196326.5000 - val_loss: 1875927.2965 - val_mse: 1875927.5000
Epoch 4/50
6392/6392 [==============================] - 0s 42us/step - loss: 2002175.2627 - mse: 2002175.6250 - val_loss: 1870844.1150 - val_mse: 1870844.0000
Epoch 5/50
6392/6392 [==============================] - 0s 42us/step - loss: 1994042.3353 - mse: 1994042.6250 - val_loss: 1858948.6113 - val_mse: 1858948.5000
Epoch 6/50
6392/6392 [==============================] - 0s 43us/step - loss: 1986265.1654 - mse: 1986265.2500 - val_loss: 1859206.7172 - val_mse: 1859207.000

In [91]:
prediction = model.predict(X_train)

In [92]:
prediction

array([[3642.6016],
       [2528.2405],
       [2549.5474],
       ...,
       [3749.5493],
       [1896.9486],
       [1140.7596]], dtype=float32)

In [93]:
Prediction_test = model.predict(df_test)

In [96]:
df_sub['Item_Outlet_Sales'] = Prediction_test
df_sub.to_csv('sample_submission_hP4II7x.csv')

In [94]:
Prediction_test

array([[1540.6882],
       [1770.7537],
       [2268.7668],
       ...,
       [2017.5748],
       [3650.553 ],
       [1471.5831]], dtype=float32)

In [95]:

for i in Prediction_test:
    if i < 0:
        print(i)